In [33]:
import sqlite3
from sqlite3 import Error
def create_connection(db_file, delete_db=False):
    
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [44]:
region_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if line.split("\t")[4] not in region_list:
            region_list.append(line.split("\t")[4])
        else: continue
region_list.sort()

for i in range(len(region_list)):
    region_list[i] = (i+1 , region_list[i]) 
    
region_list

[(1, 'British Isles'),
 (2, 'Central America'),
 (3, 'Eastern Europe'),
 (4, 'North America'),
 (5, 'Northern Europe'),
 (6, 'Scandinavia'),
 (7, 'South America'),
 (8, 'Southern Europe'),
 (9, 'Western Europe')]

In [42]:

conno = create_connection("pel.db", True)
create_table_sql = """CREATE TABLE [regn] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
create_table(conno, create_table_sql)


In [43]:
c = conno.cursor()

c.executemany('INSERT INTO regn VALUES(?, ?);',region_list)

conno.commit()
conno.close()

In [4]:
create_table_sql = """CREATE TABLE [Region] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
conn= create_connection("hello.db", True)
create_table(conn, create_table_sql)

In [5]:
def insert_region(conn, values):
    
    sql = ''' INSERT INTO Region(Region)
            VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid
    
# That the insert_region function is ready, we can start putting in values in the table as follows!
conn_norm = create_connection("hello.db")
with conn_norm:
    for region in region_list:
        insert_region(conn_norm, (region, ))

In [7]:
# Fetching all the degrees in a list
sql_statement = "SELECT Region, RegionID from Region"
regions_from_table = execute_sql_statement(sql_statement, conn)
regions_from_table

region_to_regionid_dictionary = {}

for i in range(len(regions_from_table)):
    region_to_regionid_dictionary[regions_from_table[i][0]] = regions_from_table[i][1]
    
region_to_regionid_dictionary

{'British Isles': 1,
 'Central America': 2,
 'Eastern Europe': 3,
 'North America': 4,
 'Northern Europe': 5,
 'Scandinavia': 6,
 'South America': 7,
 'Southern Europe': 8,
 'Western Europe': 9}

In [46]:
country_region_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if [line.split("\t")[3], region_to_regionid_dictionary[line.split("\t")[4]]] not in country_region_list:
            country_region_list.append([line.split("\t")[3], region_to_regionid_dictionary[line.split("\t")[4]]])
        else:
            continue

country_region_list.sort()

for i, country in enumerate(country_region_list):
    country_region_list[i] =  (i+1, country_region_list[i][0], country_region_list[i][1])
country_region_list
    

[(1, 'Argentina', 7),
 (2, 'Austria', 9),
 (3, 'Belgium', 9),
 (4, 'Brazil', 7),
 (5, 'Canada', 4),
 (6, 'Denmark', 5),
 (7, 'Finland', 6),
 (8, 'France', 9),
 (9, 'Germany', 9),
 (10, 'Ireland', 1),
 (11, 'Italy', 8),
 (12, 'Mexico', 2),
 (13, 'Norway', 6),
 (14, 'Poland', 3),
 (15, 'Portugal', 8),
 (16, 'Spain', 8),
 (17, 'Sweden', 5),
 (18, 'Switzerland', 9),
 (19, 'UK', 1),
 (20, 'USA', 4),
 (21, 'Venezuela', 7)]

In [16]:
create_table_sql = """CREATE TABLE [Country] (
[CountryID] integer not null Primary key,
[Country] Text not null,
[RegionID] integer not null,
foreign key(RegionID) References Region(RegionID)
);

"""
conn = create_connection("hello.db")

# Running the query by passing it to the `create_table` function
create_table(conn, create_table_sql)

In [17]:
def insert_country_region(conn, values):
    sql = ''' INSERT INTO Country(Country, RegionID)
            VALUES(?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# That the insert_country_region function is ready, we can start putting in values in the table as follows!
conn_norm = create_connection("hello.db")

with conn_norm:
    for country_region in country_region_list:
        try:
            insert_country_region(conn_norm, (country_region[0], country_region[1]))
        except Error:
            print(Error)